<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/useful/good_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install exchange_calendars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.1/190.1 kB 5.3 MB/s eta 0:00:00


In [2]:
!pip install finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import numpy as np
import requests
import datetime
import exchange_calendars as ecals # 개장일만
from io import StringIO
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
XKRX = ecals.get_calendar("XKRX") # 한국 코드

250일 등락률, 거래대금 90-99: 범인매매

60일 등락률 50-100, 거래대금 10-50 : 조용히 오르는 애들 (내꺼)



In [5]:
class StockList():
  def __init__(self, period=250, increased=[.9, .99], traded=[.9, .99]):
      self.period = period
      self.increased = increased
      self.traded = traded

      self.price_dic = {}

      self.start, self.today = self.get_date()
      self.df = self.get_stock_df()
      self.filtered_df = self.get_filtered_df()
      self.result_df = self.get_result_df()

  def get_date(self):
    today = datetime.date.today().strftime('%Y%m%d')
    start = (datetime.date.today() - datetime.timedelta(days=self.period)).strftime('%Y%m%d')

    if XKRX.is_session(today) == False:
      today = XKRX.previous_open(today).strftime('%Y%m%d')
    if XKRX.is_session(start) == False:
      start = XKRX.next_open(start).strftime('%Y%m%d')
    return start, today

  def get_stocks(self, market='STK'):
    data = {
      'mktId': market,
      'strtDd': self.start,
      'endDd': self.today,
      'money': '1',
      'adjStkPrc': '2',
      'adjStkPrc_check': 'Y',
      'share': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT01602'
    }
    gen_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
    gen_key = requests.post(gen_url, data=data)

    down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
    r = requests.post(down_url, data={'code':gen_key.text})
    r.encoding = 'EUC-KR'
    return pd.read_csv(StringIO(r.text))

  def get_stock_df(self):
    return pd.concat([self.get_stocks(), self.get_stocks('KSQ')]).reset_index(drop=True)

  def get_filtered_df(self):
    traded_df = self.df[(self.df['거래대금'] < self.df['거래대금'].quantile(self.traded[1])) & (self.df['거래대금'] > self.df['거래대금'].quantile(self.traded[0]))]
    increased_df = self.df[(self.df['등락률'] > self.df['등락률'].quantile(self.increased[0])) & (self.df['등락률'] < self.df['등락률'].quantile(self.increased[1]))]
    selected = pd.Series(np.intersect1d(traded_df['종목명'].values, increased_df['종목명'].values))
    return self.df[self.df['종목명'].isin(selected)].sort_values('등락률', ascending=False).head(20)

  def get_sharpe(self, df):
    change = df['Change']+1
    return change.mean()/change.std()

  def get_sortino(self, df):
    change = df['Change']+1
    return change.mean()/(change[change<1]).std()

  def get_result_df(self):
    for i in self.filtered_df['종목코드']:
      self.price_dic[i] = fdr.DataReader(str(i), self.start, self.today)

    self.filtered_df['Sharpe'] = [self.get_sharpe(self.price_dic[x]) for x in self.price_dic]
    self.filtered_df['Sortino'] = [self.get_sortino(self.price_dic[x]) for x in self.price_dic]
    return self.filtered_df.sort_values('Sharpe', ascending=False)

  def plot_df(self, start_from=0, to=5):
    fig = go.Figure()
    for i in self.price_dic:
      if (i in self.result_df['종목코드'][start_from:to].values):
        fig.add_trace(go.Scatter(x= self.price_dic[i].index, \
                                 y= self.price_dic[i]['Close']/self.price_dic[i]['Close'][0], \
                                 name= self.filtered_df[self.filtered_df['종목코드']==i]['종목명'].item()))

    fig.update_layout(title=self.today,
                      xaxis_title='Date',
                      yaxis_title='Return (%)')
    fig.show()

  def plot_stock(self, id):
    t = stocklist.price_dic[id]
    candlesticks = go.Candlestick(
        x=t.index,
        open=t['Open'],
        high=t['High'],
        low=t['Low'],
        close=t['Close'],
        showlegend=False
    )

    volume_bars = go.Bar(
        x=t.index,
        y=t['Volume'],
        showlegend=False,
        marker={
            "color": "rgba(128,128,128,0.5)",
        }
    )

    fig = go.Figure(candlesticks)
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(candlesticks, secondary_y=False
                  )
    fig.add_trace(volume_bars, secondary_y=True)
    fig.update_layout(title= self.filtered_df[self.filtered_df['종목코드']==id]['종목명'].item(), height=800)
    fig.update_xaxes(
        rangeslider_visible=True,
        rangebreaks=[
            dict(bounds=["sat", "mon"]),  # hide weekends, eg. hide sat to before mon
        ]
    )
    fig.update_yaxes(title="Price $", secondary_y=True, showgrid=True)
    fig.update_yaxes(title="Volume $", secondary_y=False, showgrid=False)
    fig.show()

In [6]:
highest = [0.99, 1]
higher = [0.9, 0.99]
longer = 500

In [7]:
stocklist = StockList()

In [16]:
t = stocklist.result_df
t

,종목코드,종목명,시작일 기준가,종료일 종가,대비,등락률,거래량,거래대금,Sharpe,Sortino
1319,086900,메디톡스,106419,250500,144081,135.39,15534865,2883770348400,26.794727,56.410355
1086,121600,나노신소재,85600,168200,82600,96.50,64254248,7629711565600,25.618363,47.546803
1277,281740,레이크머티리얼즈,4920,12200,7280,147.97,413033949,3320056687615,24.411654,56.349462
722,005070,코스모신소재,56600,169600,113000,199.65,137067949,11883588507850,23.906693,47.303781
1812,037950,엘컴텍,1195,2250,1055,88.28,1686803957,2858586859879,23.133499,52.882108
166,002710,TCC스틸,13600,38000,24400,179.41,138798799,2460796173140,22.324695,47.910440
1772,038110,에코플라스틱,2940,6670,3730,126.87,997897651,4179201573495,22.198486,54.407727
1449,000250,삼천당제약,36400,95600,59200,162.64,68643165,3739813256000,22.166622,51.873381
1734,058610,에스피지,16700,34100,17400,104.19,322645008,7981330653020,21.926296,43.251167
1201,005160,동국산업,3025,8240,5215,172.40,500068974,2812220668110,20.235201,40.618495


In [17]:
t = t[['종목코드', '종목명', '종료일 종가']].reset_index(drop=True).rename({'종료일 종가':'현재가'}, axis=1)
t.index+=1
t

,종목코드,종목명,현재가
1,086900,메디톡스,250500
2,121600,나노신소재,168200
3,281740,레이크머티리얼즈,12200
4,005070,코스모신소재,169600
5,037950,엘컴텍,2250
6,002710,TCC스틸,38000
7,038110,에코플라스틱,6670
8,000250,삼천당제약,95600
9,058610,에스피지,34100
10,005160,동국산업,8240


In [18]:
from google.colab import files
t.to_csv('df.csv') 
files.download('df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
stocklist.plot_df()